In [4]:
# Install geemap and earthengine-api if not already installed
#!pip install geemap
#!pip install earthengine-api

import ee
import geemap
import geopandas as gpd

# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project='shaurabh-urban-growth')

In [5]:
# Function to mask clouds using Sentinel-2 QA60 band
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus respectively.
    cloudBitMask = (1 << 10)
    cirrusBitMask = (1 << 11)

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

# Define your AOI (Area of Interest)
shapefile = gpd.read_file("uttarkashi_district.shp")

aoi = geemap.geopandas_to_ee(shapefile)

# Load Sentinel-2 dataset
sentinel2_2024 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2024-03-01', '2024-04-30') \
    .median()



In [6]:
# Select relevant bands and compute NDVI
bands = ['B2', 'B3', 'B4', 'B5', 'B8', 'B11', 'B12']
sentinel_2024_image = sentinel2_2024.select(bands)
ndvi = sentinel_2024_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
sentinel_2024_image = sentinel_2024_image.addBands(ndvi)



In [7]:
# Fetch land cover categories and split training/test data (Example feature collection)
landcover_categories = [1, 2, 3, 4, 5]

uk_lc_label_2024 = ee.FeatureCollection("projects/shaurabh-urban-growth/assets/uk_lc_label_2024")


# Split the data into training and test sets
def split_data(fc, ratio=0.6):
    random_column = fc.randomColumn('random')
    training_data = random_column.filter(ee.Filter.lt('random', ratio))
    test_data = random_column.filter(ee.Filter.gte('random', ratio))
    return training_data, test_data

# Splitting for 2024
training_data_combined_2024, test_data_combined_2024 = split_data(uk_lc_label_2024)


In [8]:
# Train a classifier
training_samples = sentinel_2024_image.sampleRegions(
    collection = training_data_combined_2024, # Pass the FeatureCollection as a named argument
    properties = ['lc_id'],
    scale = 10
)

classifier = ee.Classifier.smileGradientTreeBoost(100).train(
  features = training_samples,  # Pass features as a named argument
  classProperty = 'lc_id',     # Pass classProperty as a named argument
  inputProperties = bands + ['NDVI']
)

# Apply the classifier
classified_image_2024 = sentinel_2024_image.classify(classifier)

In [ ]:
# Get a list of unique class values from the classified image
class_values = classified_image_2024.reduceRegion(
    reducer= ee.Reducer.frequencyHistogram(),
    geometry= aoi,
    scale= 10,
    maxPixels= 1e13
).get('classification')

# Convert the dictionary to a list of unique class values
class_values = class_values.getInfo().keys() # Use getInfo() to get the dictionary first.

# Function to calculate area
def calculate_area(class_value):
  class_mask = classified_image_2024.eq(ee.Number(class_value))  # Cast class_value to ee.Number
  area = class_mask.multiply(ee.Image.pixelArea()).reduceRegion({
      'reducer': ee.Reducer.sum(),  # Use a dictionary for named parameters
      'geometry': classified_image_2024.geometry(),
      'scale': 10  # Adjust scale as needed
  })
  return area.get('sum')/10^6

# Calculate area for each class
class_areas = class_values.map(calculate_area)

# Print results
print("Class Areas:")
print(class_areas.getInfo())  # Use getInfo() to print the result in the console

In [13]:
# Accuracy assessment using test data
validation_samples_2024 = classified_image_2024.sampleRegions(
  collection = test_data_combined_2024, # Pass the FeatureCollection directly as the 'collection' argument
  properties = ['lc_id'],
  scale = 10
)

# Compute confusion matrix and accuracy
confusionMatrix2024 = validation_samples_2024.errorMatrix('lc_id', 'classification')

# Print accuracy metrics
print('Confusion Matrix 2024:', confusionMatrix2024.getInfo())
print('Overall Accuracy 2024:', confusionMatrix2024.accuracy().getInfo())
print('Kappa Coefficient 2024:', confusionMatrix2024.kappa().getInfo())

Confusion Matrix 2024: [[0, 0, 0, 0, 0, 0], [0, 323, 140, 50, 0, 6], [0, 120, 400, 49, 9, 0], [0, 30, 46, 1345, 16, 5], [0, 1, 3, 14, 394, 0], [0, 3, 0, 14, 0, 67]]
Overall Accuracy 2024: 0.8332784184514004
Kappa Coefficient 2024: 0.7570623417284988


In [16]:
task = ee.batch.Export.image.toDrive(image=classified_image_2024,
                                     description='classified_image_2024_new',
                                     fileNamePrefix='classified_image_2024_new',
                                     scale=10,
                                     region=aoi.geometry(),
                                     maxPixels=196979130)

task.start()

In [53]:
# Step 1: Create a mask for pixels with value 1
mask = classified_image_2024.eq(1)

# Step 2: Apply the mask to extract only the pixels with value 1
builtup_2024_image = classified_image_2024.updateMask(mask)

In [7]:
task = ee.batch.Export.image.toDrive(image=builtup_2024_image.clip(aoi),
                                     description='builtup_2024_image',
                                     scale=10
                                     )

task.start()

NameError: name 'builtup_2024_image' is not defined

In [61]:
vector_image_2024 = classified_image_2024.reduceToVectors(
    reducer=ee.Reducer.countEvery(),  # Count pixels to create polygons
    scale=10,  # Resolution of the image
    geometry=aoi,  # Area of interest for vectorization
    geometryType='polygon',
    eightConnected=False,  # Use 8-connected pixels for creating polygons
    labelProperty='classification',  # Property name for pixel values
    maxPixels=1e9  # Adjust if needed
)

# Clip the vectorized image to the AOI
vector_image_2024_final = vector_image_2024.filterBounds(aoi)

In [62]:
task = ee.batch.Export.table.toDrive(
    collection=vector_image_2024_final,
    description='classified_image_2024_vector',
    fileNamePrefix='classified_image_2024_vector',
    folder='GEE_exports',  # Optional: specify a folder in Google Drive
    fileFormat='SHP'  # Export format as Shapefile
)

# Start the export task
task.start()

print("Export task started.")

Export task started.


In [ ]:
import ee
import geemap

# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project='shaurabh-urban-growth')


In [5]:
import geopandas as gpd
shapefile = gpd.read_file("uttarkashi_district.shp")

aoi = geemap.geopandas_to_ee(shapefile)

In [15]:
image_2020= ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2020')
built_2020 = image_2020.select('built_surface')


In [14]:
task = ee.batch.Export.image.toDrive(image=built_2020,
                                     description='built_2020_image',
                                     scale=100,
                                     region=aoi.geometry(),
                                     maxPixels=172891356
                                     )

task.start()

In [17]:
image_2025= ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2025')
built_2025 = image_2020.select('built_surface')

task = ee.batch.Export.image.toDrive(image=built_2025,
                                     description='built_2025_image',
                                     scale=100,
                                     region=aoi.geometry(),
                                     maxPixels=172891356
                                     )

task.start()


In [21]:
# Set the time range for 2024.
startDate = '2024-03-01'
endDate = '2024-04-30'

# Filter the Dynamic World dataset for the given date range and AOI.
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
  .filter(ee.Filter.date(startDate, endDate)) \
  .filterBounds(aoi)

#print('Dynamic World Collection:', dw.size().getInfo())
# Select the 'label' band for classification and create a mode composite.
classification = dw.select('label')
dwComposite = classification.reduce(ee.Reducer.mode())

# Visualize the land cover classification.
dwVisParams = {
    'min': 0,
    'max': 8,
    'palette': ['#419BDF', '#397D49', '#88B053', '#7A87C6',
                '#E49635', '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1']
}

# Add the land cover map to the interactive map.
Map = geemap.Map()
Map.addLayer(dwComposite.clip(aoi), dwVisParams, 'Land Cover Map 2020')
Map
# Export the land cover map as a GeoTIFF to Google Drive.
export_task = ee.batch.Export.image.toDrive(**{
    'image': dwComposite.clip(aoi),
    'description': 'DW_LandCoverMap_2024',
    'scale': 10,  # Adjust scale according to your needs
    'region': aoi.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13
})

# Start the export task.
export_task.start()
print('Exporting land cover map to Google Drive...')
# Define the probability bands in the Dynamic World dataset.
probabilityBands = ['water', 'trees', 'grass', 'flooded_vegetation',
                    'crops', 'shrub_and_scrub', 'built',
                    'bare', 'snow_and_ice']

# Select probability bands and calculate the average probability for each band.
probabilityCol = dw.select(probabilityBands)
meanProbability = probabilityCol.reduce(ee.Reducer.mean())

# Set the projection to ensure appropriate scale.
projection = ee.Projection('EPSG:3857').atScale(10)
meanProbability = meanProbability.setDefaultProjection(projection)

# Export the mean probability layer to Google Drive.
export_task = ee.batch.Export.image.toDrive(**{
    'image': meanProbability.clip(aoi),
    'description': 'DW_MeanProbability_Layer_2024',
    'scale': 10,  # Adjust scale to the desired resolution (meters per pixel)
    'region': aoi.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13  # Ensure this value is high enough for large images
})

# Start the export task.
export_task.start()
print('Exporting mean probability layer to Google Drive...')


Exporting land cover map to Google Drive...
Exporting mean probability layer to Google Drive...


In [ ]:
# Set the time range for 2018.
startDate = '2018-03-01'
endDate = '2018-04-30'

# Filter the Dynamic World dataset for the given date range and AOI.
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
  .filter(ee.Filter.date(startDate, endDate)) \
  .filterBounds(aoi)

#print('Dynamic World Collection:', dw.size().getInfo())
# Select the 'label' band for classification and create a mode composite.
classification = dw.select('label')
dwComposite = classification.reduce(ee.Reducer.mode())

# Visualize the land cover classification.
dwVisParams = {
    'min': 0,
    'max': 8,
    'palette': ['#419BDF', '#397D49', '#88B053', '#7A87C6',
                '#E49635', '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1']
}

# Add the land cover map to the interactive map.
Map = geemap.Map()
Map.addLayer(dwComposite.clip(aoi), dwVisParams, 'Land Cover Map 2020')
Map
# Export the land cover map as a GeoTIFF to Google Drive.
export_task = ee.batch.Export.image.toDrive(**{
    'image': dwComposite.clip(aoi),
    'description': 'DW_LandCoverMap_2018',
    'scale': 10,  # Adjust scale according to your needs
    'region': aoi.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13
})

# Start the export task.
export_task.start()
print('Exporting land cover map to Google Drive...')
# Define the probability bands in the Dynamic World dataset.
probabilityBands = ['water', 'trees', 'grass', 'flooded_vegetation',
                    'crops', 'shrub_and_scrub', 'built',
                    'bare', 'snow_and_ice']

# Select probability bands and calculate the average probability for each band.
probabilityCol = dw.select(probabilityBands)
meanProbability = probabilityCol.reduce(ee.Reducer.mean())

# Set the projection to ensure appropriate scale.
projection = ee.Projection('EPSG:3857').atScale(10)
meanProbability = meanProbability.setDefaultProjection(projection)

# Export the mean probability layer to Google Drive.
export_task = ee.batch.Export.image.toDrive(**{
    'image': meanProbability.clip(aoi),
    'description': 'DW_MeanProbability_Layer_2018',
    'scale': 10,  # Adjust scale to the desired resolution (meters per pixel)
    'region': aoi.geometry(),
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13  # Ensure this value is high enough for large images
})

# Start the export task.
export_task.start()
print('Exporting mean probability layer to Google Drive...')


Exporting land cover map to Google Drive...
Exporting mean probability layer to Google Drive...
